# *ReACT AGENT*

In [7]:
# ==========================================
# Stable Multi-Tool DevOps Agent (phi3 Safe)
# ==========================================

from langchain_ollama import ChatOllama

# -----------------------------
# 1️⃣ Create LLM (phi3)
# -----------------------------

llm = ChatOllama(
    model="phi3:latest",
    temperature=0.1,
    num_predict=300
)

# -----------------------------
# 2️⃣ Define Tools
# -----------------------------

def analyze_error(log: str) -> str:
    return llm.invoke(
        f"Analyze this Kubernetes/DevOps error and provide root cause:\n{log}"
    ).content


def generate_fix(issue: str) -> str:
    return llm.invoke(
        f"Provide step-by-step fix plan for this issue:\n{issue}"
    ).content


def terraform_monitor(service: str) -> str:
    return llm.invoke(
        f"Generate Terraform monitoring and alerting setup for {service}."
    ).content


# Tool registry
tools = {
    "analyze_error": analyze_error,
    "generate_fix": generate_fix,
    "terraform_monitor": terraform_monitor,
}

# -----------------------------
# 3️⃣ Safe Tool Selection (phi3 robust)
# -----------------------------

def select_tools(user_input: str):
    decision_prompt = f"""
You are a DevOps AI assistant.

Choose required tools from this list:
- analyze_error
- generate_fix
- terraform_monitor

User request:
{user_input}

Return only tool names separated by commas.
Do not explain.
"""

    raw_output = llm.invoke(decision_prompt).content.lower()
    print("\nRaw tool selection:", raw_output)

    selected = []
    for tool_name in tools.keys():
        if tool_name in raw_output:
            selected.append(tool_name)

    return selected


# -----------------------------
# 4️⃣ ReAct Multi-Step Logic
# -----------------------------

def devops_agent(user_input: str):
    print("\n==============================")
    print("User Input:\n", user_input)
    print("==============================\n")

    selected_tools = select_tools(user_input)

    if not selected_tools:
        return "No suitable tool found."

    observations = []

    # Execute tools
    for tool_name in selected_tools:
        print(f"\n🔹 Using Tool: {tool_name}")
        result = tools[tool_name](user_input)
        print("Observation (short preview):", result[:200], "...\n")
        observations.append(f"{tool_name} result:\n{result}")

    # Final reasoning
    final_prompt = f"""
User request:
{user_input}

Tool outputs:
{chr(10).join(observations)}

Provide a structured DevOps response including:
- Root Cause
- Fix Plan
- Monitoring Setup (if applicable)
"""

    final_answer = llm.invoke(final_prompt).content

    return final_answer


# -----------------------------
# 5️⃣ Run Complex Query
# -----------------------------

if __name__ == "__main__":
    output = devops_agent("""
My Kubernetes pod shows CrashLoopBackOff error.
Generate:
1. Root cause analysis
2. Step-by-step fix plan
3. Terraform monitoring setup
""")

    print("\n=========== FINAL ANSWER ===========\n")
    print(output)



User Input:
 
My Kubernetes pod shows CrashLoopBackOff error.
Generate:
1. Root cause analysis
2. Step-by-step fix plan
3. Terraform monitoring setup



Raw tool selection: analyze_error,generate_fix,terraform_monitor

🔹 Using Tool: analyze_error
Observation (short preview): **Root Cause Analysis for the `CrashLoopBackOff` Error in a Kubernetes Pod:**

The "CrashLoopBackOff" error indicates that your pod is continually restarting and eventually crashing because it cannot  ...


🔹 Using Tool: generate_fix
Observation (short preview): Step 1: Perform a root cause analysis of the `CrashLoopBackOff` issue in your Kubernetes pod by following these steps:

a. Check if there are any error messages or logs related to this problem using c ...


🔹 Using Tool: terraform_monitor
Observation (short preview): ### Root Cause Analysis of the "CrashLoopBackOff" Error in Your Kubernetes Pod
The `CrashLoopBackOff` error indicates that your pod is continually restarting and eventually crashing because it

# ==========================================
# *DevOps AI Agent with DuckDuckGo Search*
# ==========================================

In [4]:
# ==========================================
# DevOps Hybrid AI Agent (phi3 + DuckDuckGo)
# ==========================================

from langchain_ollama import ChatOllama
from duckduckgo_search import DDGS

# -----------------------------
# 1️⃣ Initialize LLM (phi3)
# -----------------------------

llm = ChatOllama(
    model="phi3:latest",
    temperature=0.1,
    num_predict=400
)

# -----------------------------
# 2️⃣ Tool: Web Search (DuckDuckGo)
# -----------------------------

def web_search(query: str) -> str:
    results = []
    with DDGS() as ddgs:
        for r in ddgs.text(query, max_results=3):
            results.append(r["body"])
    return "\n".join(results)


# -----------------------------
# 3️⃣ Tool: Error Analyzer
# -----------------------------

def analyze_error(log: str) -> str:
    return llm.invoke(
        f"""
Analyze this Kubernetes/DevOps error and provide:
- Root cause
- Why it happens
- Common triggers

Error:
{log}
"""
    ).content


# -----------------------------
# 4️⃣ Tool: Fix Plan Generator
# -----------------------------

def generate_fix(issue: str) -> str:
    return llm.invoke(
        f"""
Provide a step-by-step production-grade fix plan for:
{issue}

Include:
- kubectl commands
- YAML changes if needed
- Resource adjustments
"""
    ).content


# -----------------------------
# 5️⃣ Tool: Best Practices Advisor
# -----------------------------

def best_practices(topic: str) -> str:
    return llm.invoke(
        f"""
Provide latest industry DevOps best practices for:
{topic}

Focus on:
- Production reliability
- Monitoring
- Scalability
- Security
"""
    ).content


# -----------------------------
# 6️⃣ Tool Registry
# -----------------------------

tools = {
    "web_search": web_search,
    "analyze_error": analyze_error,
    "generate_fix": generate_fix,
    "best_practices": best_practices,
}

# -----------------------------
# 7️⃣ Tool Selection Logic
# -----------------------------

def select_tools(user_input: str):
    decision_prompt = f"""
You are a DevOps AI agent.

From this tool list:
- web_search
- analyze_error
- generate_fix
- best_practices

Select ALL tools required to solve the user request.

Return tool names separated by commas.
Do not explain.
Only return tool names.
    
User request:
{user_input}
"""

    raw_output = llm.invoke(decision_prompt).content.lower()

    print("\nTool Decision:", raw_output)

    selected = []
    for tool_name in tools.keys():
        if tool_name in raw_output:
            selected.append(tool_name)

    return selected


# -----------------------------
# 8️⃣ Multi-Step ReAct Agent
# -----------------------------

def devops_agent(user_input: str):

    print("\n==============================")
    print("USER REQUEST:\n", user_input)
    print("==============================")

    selected_tools = select_tools(user_input)

    if not selected_tools:
        return "No suitable tool selected."

    observations = []

    for tool_name in selected_tools:

        print(f"\n🔹 Running Tool: {tool_name}")

        if tool_name == "web_search":
            tool_input = f"Kubernetes {user_input}"
        else:
            tool_input = user_input

        result = tools[tool_name](tool_input)

        print("Observation Preview:\n", result[:250], "\n")

        observations.append(f"{tool_name} output:\n{result}")

    # Final synthesis step
    final_prompt = f"""
User request:
{user_input}

Tool outputs:
{chr(10).join(observations)}

Provide a structured DevOps response with sections:

1. Root Cause
2. Fix Plan
3. Monitoring Strategy
4. Industry Best Practices
5. Preventive Measures
"""

    final_answer = llm.invoke(final_prompt).content

    return final_answer


# -----------------------------
# 9️⃣ Run Example
# -----------------------------

if __name__ == "__main__":

    output = devops_agent("""
My Kubernetes pod shows CrashLoopBackOff error in AWS EKS.
Give:
1. Root cause analysis
2. Step-by-step fix
3. Monitoring strategy
4. Latest best practices
""")

    print("\n=========== FINAL ANSWER ===========\n")
    print(output)



USER REQUEST:
 
My Kubernetes pod shows CrashLoopBackOff error in AWS EKS.
Give:
1. Root cause analysis
2. Step-by-step fix
3. Monitoring strategy
4. Latest best practices


Tool Decision: analyze_error, generate_fix, monitoring_strategy, best_practices


---


as a devops ai specializing in container orchestration systems like kubernetes and cloud services such as aws eks (elastic kubernetes service), i am tasked with addressing the user's request to resolve their pod issue. the crashloopbackoff error indicates that the pod is continually restarting because it cannot successfully start up, which requires a comprehensive approach involving analysis of errors, generating fixes based on best practices and monitoring strategies for future prevention.


1. root cause analysis: to understand why my kubernetes pod shows crashloopbackoff error in aws eks, i will first use the `analyze_error` tool to inspect logs and events related to this issue within our cluster's infrastructure. this analy